# Pyrovelocity Colab user notebook template

<a target="_blank" href="https://colab.research.google.com/github/pinellolab/pyrovelocity/blob/520-sum/docs/source/notebooks/pyrovelocity_colab_user_template.ipynb">
  <img 
    src="https://colab.research.google.com/assets/colab-badge.svg" 
    alt="Open In Colab"
    width="109" height="20"/>
</a> <a target="_blank" href="https://nbviewer.jupyter.org/github/pinellolab/pyrovelocity/blob/520-sum/docs/source/notebooks/pyrovelocity_colab_user_template.ipynb">
  <img 
    src="https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.png"
    alt="Render with nbviewer" 
    width="109" height="20"/>
</a>

## Setup environment

Installation should take less than **5 minutes**. 
It involves

- (~1m) installing condacolab to access conda packages from colab
- (~4m) installing pyrovelocity from conda-forge with mamba

The simplest way to complete it is to run the `Setup environment` section, wait for the kernel to restart, and then run the same section again:

- **fold** this `Setup environment` section above
- **click the play button** underneath the section name to run the whole section for the first time (installs condacolab)
- **wait** for the kernel to restart 
  - **ignore** expected notice in bottom left of Colab UI
    - `Your session crashed for an unknown reason. View runtime logs`
  - **ignore** `SystemExit` output in the `Install condacolab` subsection below
- **refold** the `Setup environment` section (`SystemExit` / kernel restart will unfold it)
- **click the same play button** as above to run the `Setup environment` section for a second time (installs pyrovelocity from conda-forge)

Otherwise, the cells below can be executed manually.
In either case, this section can be folded away after installation is complete.

If you need to edit the **version number**, please see below.

### Install conda in colab

This first stage will download and install [condacolab](https://github.com/conda-incubator/condacolab/tree/0.1.x). This usually takes less than **30 seconds**. The runtime will then automatically restart. After this you can execute "Run all" to complete installation or proceed linearly below if you have added additional content you do not want to run all at once.

#### Define functions to manage installation of condacolab

In [ ]:
import importlib.util
import subprocess
import sys


def is_module_available(module_name: str):
    return importlib.util.find_spec(module_name) is not None


def install_package(package_name: str):
    """
    Install a package using pip. This is similar to cell magic
    `!pip install package_name`, but in python code for compatibility
    outside jupyter.

    Args:
        package_name (str): Name of the package to install.
    """
    process = subprocess.Popen(
        ["pip", "install", "-q", package_name],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
    )
    for line in process.stdout:
        print(line, end="")
    for line in process.stderr:
        print(line, end="")
    process.wait()


def setup_condacolab():
    if is_module_available("condacolab"):
        try:
            import condacolab

            condacolab.check()
        except (ImportError, AssertionError):
            print("condacolab is not successfully installed")
            sys.exit()
    else:
        print("Installing condacolab...")
        install_package("condacolab")
        try:
            import condacolab

            condacolab.install_mambaforge()
            print(
                "\nThe kernel needs to restart in order to use condacolab.\n"
                "Please run this cell again.\n"
            )
            sys.exit()
        except (ImportError, AssertionError):
            print("Failed to install condacolab properly.")
            sys.exit()


#### Install condacolab

In [ ]:
import os

IN_COLAB = is_module_available("google.colab")

if IN_COLAB:
    colab_release_tag = os.getenv("COLAB_RELEASE_TAG", None)
    print(f"Google Colab release: {colab_release_tag}")
    setup_condacolab()
else:
    print("This notebook is probably not running in Google Colab")

### Install library and dependencies

The second stage installs the library from [conda-forge](https://conda-forge.org/packages/). See [conda-forge/packages/pyrovelocity](https://anaconda.org/conda-forge/pyrovelocity/files) for available versions, build strings, and build numbers. For example in

> `noarch/pyrovelocity-0.2.0b11.dev1-pyhff70e4c_0.conda`

`0.2.0b11.dev1` is the version
`pyhff70e4c` is the build string and
`0` is the build number. All three need to be specified to determine a unique package version and associated installation. We generally intend to keep the defaults below close to the latest stable version. This usually takes less than **3 minutes**.

In [ ]:
%%bash

set -euo pipefail

printf "Checking for package managers...\n"

PACKAGE_NAME="pyrovelocity"
PACKAGE_VERSION="0.2.0b11.dev1"
CONDA_BUILD_STRING="pyhff70e4c"
CONDA_BUILD_NUMBER="0"
CONDA_CHANNEL_LABEL="pyrovelocity_dev"

PACKAGE_SPEC="conda-forge/label/\
$CONDA_CHANNEL_LABEL::\
$PACKAGE_NAME=$PACKAGE_VERSION=$CONDA_BUILD_STRING"_"$CONDA_BUILD_NUMBER"

USE_COLOR=true
if [ -t 1 ]; then
    USE_COLOR=true
else
    USE_COLOR=false
fi

BLUE="\033[0;34;1m"
BOLD="\033[1m"
NO_COLOR="\033[0m"

print_with_color() {
    local format=$1
    shift
    if [ "$USE_COLOR" = true ]; then
        printf "$format" "$@"
    else
        printf "$(echo "$format" | sed 's/\x1b\[[0-9;]*m//g')" "$@"
    fi
}

command_exists() {
    command -v "$1" >/dev/null 2>&1
}

package_is_installed() {
    local manager="$1"
    local version_spec_regex="\
${PACKAGE_NAME}.*${PACKAGE_VERSION}.*\
${CONDA_BUILD_STRING}_${CONDA_BUILD_NUMBER}"
    if $manager list | grep -E "$version_spec_regex" > /dev/null; then
        return 0
    else
        return 1
    fi
}

install_package() {
    local manager="$1"
    printf "Using manager ${BLUE}'%s'${NO_COLOR}...\n" "$manager"
    printf "\n\
    Checking if\n\n\
    ${BOLD}%s${NO_COLOR}\n\
    is installed...\n" "$PACKAGE_SPEC"
    if package_is_installed "$manager"; then
        printf "\n\
        Package ${BLUE}'%s'${NO_COLOR}\n\
        with version ${BLUE}'%s'${NO_COLOR}\n\
        is already installed.\n" "$PACKAGE_NAME" "$PACKAGE_VERSION"
    else
        printf "\n\
        Installing ${BLUE}'%s'${NO_COLOR}\n\
        from conda-forge using ${BLUE}'%s'${NO_COLOR}...\n"\
        "$PACKAGE_NAME" "$manager"
        set -x
        $manager install "$PACKAGE_SPEC" > "${PACKAGE_NAME}_install.log" 2>&1
        { set +x; } 2>/dev/null
    fi
}

if command_exists mamba; then
    manager="mamba"
elif command_exists conda; then
    manager="conda"
else
    printf "\n\
    Neither ${BOLD}mamba${NO_COLOR} nor ${BOLD}conda${NO_COLOR} is available.\n\
    Please install mamba or conda and try again.\n\
    Installation ${BOLD}failed${NO_COLOR}.\n"
    exit 1
fi

install_package "$manager"

printf "\nInstallation ${BOLD}complete${NO_COLOR}.\n\n"

print_with_color "\n\
Installed ${BLUE}conda${NO_COLOR} packages\
 containing ${BOLD}'%s'${NO_COLOR}:\n" "$PACKAGE_NAME"
$manager list | grep "$PACKAGE_NAME"
print_with_color "\n\
Installed ${BLUE}pip${NO_COLOR} packages\
 containing ${BOLD}'%s'${NO_COLOR}:\n" "$PACKAGE_NAME"
pip list | grep "$PACKAGE_NAME"

print_with_color "\n\
If ${BOLD}conda <-> pip${NO_COLOR} versions are not consistent,\n\
the package may function as expected, but the\n\
${BOLD}python${NO_COLOR} package source likely contains a\n\
different ${BOLD}version string${NO_COLOR} from that used in\n\
building the ${BOLD}conda${NO_COLOR} package.\n"

If you want to see the full list of installed packages run `!cat pyrovelocity_install.log`, open it in a text editor, or run `!mamba list`.

### Check installation

If installation was successful, the following commands should print the location of the `__init__.py` file for the pyrovelocity package and the currently installed version.

In [ ]:
import pyrovelocity
print(pyrovelocity.__file__)
print(pyrovelocity.__version__)

This is the same for the pyro package in case there was an issue with pyrovelocity install that did not affect another package.

In [ ]:
import pyro
print(pyro.__file__)
print(pyro.__version__)

Please refer to the [docs](https://pinellolab.github.io/pyrovelocity) for tutorials and usage information.

---

## Analysis

In [ ]:
help(pyrovelocity) # ?pyrovelocity # to open in side panel tab for reference